# Teoría de funcionales de la densidad (DFT)

La teoría de los funcionales de la densidad se basa en que la densidad electrónica contiene toda la información del sistema, por lo que se puede extraer de ella la energía mediante un funcional

$$
E[\rho] = T_s[\rho] + U[\rho] + V_{nuc}[\rho] + E_{xc}[\rho]
$$

Donde
- $E[\rho]$ es la energía del sistema 
- $T_s[\rho]$ es la energía cinética de Kohn-Sham
- $U[\rho]$ es la energía de Hartree
- $V[\rho]$ es la interacción núcleo electrón
- $E_{xc} [\rho]$ es la energía de intercambio correlación

En la formulación de Kohn y Sham, la densidad electrónica ($\rho(r)$) se calcula a partir de los orbitales de Kohn-Sham ($\psi^{KS}_i(r)$)

$$
\rho(r) = \sum_i^{N} |\psi^{KS}_i(r)|^2
$$


```{warning}
Los teoremas de Hohenberg y Kohn, y Kohn y Sham prueban que existe un funcional universal que conecta a la densidad electrónica con la energía del sistema, pero este funcional no se conoce, particularmente por la parte de $E_{xc}[\rho]$. Se han realizado diversas propuestas de como construir este funcional, por lo que en la práctica existen cientos de funcionales de DFT.
```

```{note}
No existe el mejor funcional, sino que depende del sistema químico que se esté estudiando.
```

En general los funcionales aproximan de diferente manera el intercambio y la correlación, dependiendo de como se aproxime el funcional, estos se pueden clasificar en diferentes categorías. La clasificación fue propuesta por Perdew y se conoce como la `escalera de Jacob`.

```{warning}
Evaluar la contribución del funcional de intercambio-correlación requiere de realizar integración numérica. Esto se hace con un mallado entorno a la molécula. Existen diversos esquemas para colocar los puntos en el mallado, y para seleccionar cuantos puntos poner, estos puede cambiar de un software a otro, e incluso entre diferentes versiones de un mismo software.
```

**Importe psi4**

In [1]:
#Importe psi4

In [2]:
import psi4

Para ejemplificar el uso de estos funcionales, declare la molécula de agua.

```
h2o = psi4.geometry("""
0 1
O    0.0000    0.0000    0.1173
H    0.0000    0.7572   -0.4692
H    0.0000   -0.7572   -0.4692 
units angstrom
""")
```

In [3]:
# h2o

In [4]:
h2o = psi4.geometry("""
0 1
O    0.0000    0.0000    0.1173
H    0.0000    0.7572   -0.4692
H    0.0000   -0.7572   -0.4692 
units angstrom
""")

## Aproximación Local de la Densidad (LDA)

Fue una de las primeras aproximaciones y ya ha sido superada. En este caso, el intercambio se calcula mediante

$$
E_x^{LDA} = -\frac{3}{4} \left( \frac{3}{\pi} \right)^{1/3} \int \rho^{4/3} (r) dr 
$$

y la correlación se calcula mediante el funcional de Vosko, Wilk y Nusair
$$
E_c^{LDA} = \int \varepsilon_c^{VWN} dr
$$

Haga un cálculo de energía con SVWN y la base 6-311G con la siguiente instrucción
```
psi4.energy('SVWN/6-311G')
```

In [5]:
# SVWN

In [6]:
psi4.energy('SVWN/6-311G')


*** tstart() called on BlauweDraak
*** at Thu Jun  2 11:07:38 2022

   => Loading Basis Set <=

    Name: 6-311G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   126 file /home/jebarrios/anaconda3/envs/QCI/share/psi4/basis/6-311g.gbs 
    atoms 2-3 entry H          line    21 file /home/jebarrios/anaconda3/envs/QCI/share/psi4/basis/6-311g.gbs 


         ---------------------------------------------------------
                                   SCF
            by Justin Turney, Rob Parrish, Andy Simmonett
                             and Daniel Smith
                              RKS Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c2v
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass       
    ------------   --

   => Loading Basis Set <=

    Name: (6-311G AUX)
    Role: JKFIT
    Keyword: DF_BASIS_SCF
    atoms 1   entry O          line   229 file /home/jebarrios/anaconda3/envs/QCI/share/psi4/basis/cc-pvtz-jkfit.gbs 
    atoms 2-3 entry H          line    51 file /home/jebarrios/anaconda3/envs/QCI/share/psi4/basis/cc-pvtz-jkfit.gbs 

  ==> Pre-Iterations <==

   -------------------------------------------------------
    Irrep   Nso     Nmo     Nalpha   Nbeta   Ndocc  Nsocc
   -------------------------------------------------------
     A1        10      10       0       0       0       0
     A2         0       0       0       0       0       0
     B1         3       3       0       0       0       0
     B2         6       6       0       0       0       0
   -------------------------------------------------------
    Total      19      19       5       5       5       0
   -------------------------------------------------------

  ==> Integral Setup <==

  DFHelper Memory: AOs need 0.001

-76.04571335924186

	user time   =       0.49 seconds =       0.01 minutes
	system time =       0.01 seconds =       0.00 minutes
	total time  =          0 seconds =       0.00 minutes
Total time:
	user time   =       0.49 seconds =       0.01 minutes
	system time =       0.01 seconds =       0.00 minutes
	total time  =          0 seconds =       0.00 minutes


## Aproximación de Gradientes Generalizados (GGA)

Esto calcula la energía con base en la densidad electrónica y su gradiente

$$
E_{xc}^{GGA} = -\int \varepsilon_{xc}^{GGA} (\rho,\nabla \rho) dr
$$

Para ello separa los funcionales en intercambio y correlación.

$$
\varepsilon_{xc}^{GGA} = \varepsilon_{x}^{GGA} + \varepsilon_{c}^{GGA}
$$

Algunos funcionales de intercambio GGA son
- PWx86: Perdew-Wang 1986
- B88: Becke 1988
- PWx91: Perdew-Wang 1991
- PBE: Perdew-Burke-Ernzerhof
    
Algunos funcionales de correlación GGA son
- LYP: Lee-Yang-Parr
- Pc86: Perdew 1986
- PWc91: Perdew-Wang 1991
- PBE: Perdew-Burke-Ernzerhof
    
La combinación de estos funcionales genera los funcionales GGA. **Haga un cálculo de energía con PBE y la base 6-311G con la siguiente instrucción**
```
psi4.energy('PBE/6-311G')
```

In [7]:
psi4.energy('pbe/6-311G')


*** tstart() called on BlauweDraak
*** at Thu Jun  2 11:07:38 2022

   => Loading Basis Set <=

    Name: 6-311G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   126 file /home/jebarrios/anaconda3/envs/QCI/share/psi4/basis/6-311g.gbs 
    atoms 2-3 entry H          line    21 file /home/jebarrios/anaconda3/envs/QCI/share/psi4/basis/6-311g.gbs 


         ---------------------------------------------------------
                                   SCF
            by Justin Turney, Rob Parrish, Andy Simmonett
                             and Daniel Smith
                              RKS Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c2v
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass       
    ------------   --

  ==> Integral Setup <==

  DFHelper Memory: AOs need 0.001 [GiB]; user supplied 0.366 [GiB]. Using in-core AOs.

  ==> MemDFJK: Density-Fitted J/K Matrices <==

    J tasked:                   Yes
    K tasked:                    No
    wK tasked:                   No
    OpenMP threads:               1
    Memory (MB):                375
    Algorithm:                 Core
    Schwarz Cutoff:           1E-12
    Mask sparsity (%):       0.0000
    Fitting Condition:        1E-12

   => Auxiliary Basis Set <=

  Basis Set: (6-311G AUX)
    Blend: CC-PVTZ-JKFIT
    Number of shells: 45
    Number of basis function: 139
    Number of Cartesian functions: 166
    Spherical Harmonics?: true
    Max angular momentum: 4

  Minimum eigenvalue in the overlap matrix is 4.3275028627E-02.
  Using Symmetric Orthogonalization.

  SCF Guess: Superposition of Atomic Densities via on-the-fly atomic UHF.

  ==> Iterations <==

                           Total Energy        Delta E     RMS |[F,P]|

   

@DF-RKS iter   0:   -76.31315107372512   -7.63132e+01   1.30451e-01 
   @DF-RKS iter   1:   -76.24493028291617    6.82208e-02   3.99406e-02 
   @DF-RKS iter   2:   -76.04807225996659    1.96858e-01   7.22912e-02 DIIS
   @DF-RKS iter   3:   -76.32764284494797   -2.79571e-01   1.23925e-03 DIIS
   @DF-RKS iter   4:   -76.32774534679760   -1.02502e-04   1.66898e-04 DIIS
   @DF-RKS iter   5:   -76.32774681260878   -1.46581e-06   1.16320e-05 DIIS
   @DF-RKS iter   6:   -76.32774681997822   -7.36944e-09   3.75999e-07 DIIS

  ==> Post-Iterations <==

    Orbital Energies [Eh]
    ---------------------

    Doubly Occupied:                                                      

       1A1   -18.723192     2A1    -0.934053     1B2    -0.480903  
       3A1    -0.302844     1B1    -0.236425  

    Virtual:                                                              

       4A1     0.003413     2B2     0.084152     3B2     0.344734  
       5A1     0.378494     2B1     0.719640     6A1     0.740

-76.32774681997822

## Aproximación meta-GGA

Los funcionales meta-GGA usan la densidad electrónica, el gradiente de la densidad electrónica, y el laplaciano de la densidad electrónica.

$$
E_{xc}^{meta-GGA} = -\int \varepsilon_{xc}^{meta-GGA} (\rho,\nabla \rho,\nabla^2 \rho) dr
$$

Algunos ejemplos son:
- B95: Becke 1995
- TPSS: Tau-Perdew-Staroverov-Scuseria

**Haga un cálculo de energía con TPSS y la base 6-311G con la siguiente instrucción**
```
psi4.energy('TPSS/6-311G')
```

In [8]:
# TPSS

In [9]:
psi4.energy('TPSS/6-311G')


*** tstart() called on BlauweDraak
*** at Thu Jun  2 11:07:39 2022

   => Loading Basis Set <=

    Name: 6-311G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   126 file /home/jebarrios/anaconda3/envs/QCI/share/psi4/basis/6-311g.gbs 
    atoms 2-3 entry H          line    21 file /home/jebarrios/anaconda3/envs/QCI/share/psi4/basis/6-311g.gbs 


         ---------------------------------------------------------
                                   SCF
            by Justin Turney, Rob Parrish, Andy Simmonett
                             and Daniel Smith
                              RKS Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c2v
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass       
    ------------   --

   => Loading Basis Set <=

    Name: (6-311G AUX)
    Role: JKFIT
    Keyword: DF_BASIS_SCF
    atoms 1   entry O          line   229 file /home/jebarrios/anaconda3/envs/QCI/share/psi4/basis/cc-pvtz-jkfit.gbs 
    atoms 2-3 entry H          line    51 file /home/jebarrios/anaconda3/envs/QCI/share/psi4/basis/cc-pvtz-jkfit.gbs 

  ==> Pre-Iterations <==

   -------------------------------------------------------
    Irrep   Nso     Nmo     Nalpha   Nbeta   Ndocc  Nsocc
   -------------------------------------------------------
     A1        10      10       0       0       0       0
     A2         0       0       0       0       0       0
     B1         3       3       0       0       0       0
     B2         6       6       0       0       0       0
   -------------------------------------------------------
    Total      19      19       5       5       5       0
   -------------------------------------------------------

  ==> Integral Setup <==

  DFHelper Memory: AOs need 0.001

@DF-RKS iter   0:   -76.39779118367051   -7.63978e+01   1.30232e-01 
   @DF-RKS iter   1:   -76.33855949560252    5.92317e-02   3.90136e-02 
   @DF-RKS iter   2:   -76.18341715869511    1.55142e-01   6.63527e-02 DIIS
   @DF-RKS iter   3:   -76.41716011962504   -2.33743e-01   1.08966e-03 DIIS
   @DF-RKS iter   4:   -76.41723395980274   -7.38402e-05   1.30817e-04 DIIS
   @DF-RKS iter   5:   -76.41723476471714   -8.04914e-07   1.07439e-05 DIIS
   @DF-RKS iter   6:   -76.41723477132335   -6.60621e-09   6.64971e-07 DIIS

  ==> Post-Iterations <==

    Orbital Energies [Eh]
    ---------------------

    Doubly Occupied:                                                      

       1A1   -18.863827     2A1    -0.956505     1B2    -0.487222  
       3A1    -0.312034     1B1    -0.244459  

    Virtual:                                                              

       4A1     0.011915     2B2     0.091731     3B2     0.352344  
       5A1     0.386412     2B1     0.735694     6A1     0.758

-76.41723477132335

## Funcionales Híbridos

Mezclan un funcional de intercambio con el `intercambio de Hartree-Fock` en alguna proporción.

Algunos ejemplos de estos funcionales son:

- B3LYP
- PBE0
- M05-2X y M06-2X
- TPSSh

**Haga un cálculo de energía con M06-2X y la base 6-311G con la siguiente instrucción**
```
psi4.energy('M062X/6-311G')
```

In [10]:
#M062X

In [11]:
psi4.energy('M062X/6-311G')


*** tstart() called on BlauweDraak
*** at Thu Jun  2 11:07:40 2022

   => Loading Basis Set <=

    Name: 6-311G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   126 file /home/jebarrios/anaconda3/envs/QCI/share/psi4/basis/6-311g.gbs 
    atoms 2-3 entry H          line    21 file /home/jebarrios/anaconda3/envs/QCI/share/psi4/basis/6-311g.gbs 


         ---------------------------------------------------------
                                   SCF
            by Justin Turney, Rob Parrish, Andy Simmonett
                             and Daniel Smith
                              RKS Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c2v
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass       
    ------------   --

   0.424968  
       5A1     0.459894     2B1     0.832832     6A1     0.853903  
       4B2     0.959332     7A1     1.225444     8A1     2.173262  
       5B2     2.212145     3B1     4.881181     9A1     4.985629  
       6B2     5.094104    10A1    49.850269  

    Final Occupation by Irrep:
             A1    A2    B1    B2 
    DOCC [     3,    0,    1,    1 ]

  Energy converged.

  @DF-RKS Final Energy:   -76.37840759959782

   => Energetics <=

    Nuclear Repulsion Energy =              9.1895337224728184
    One-Electron Energy =                -123.0465201037716980
    Two-Electron Energy =                  41.9684554030333601
    DFT Exchange-Correlation Energy =      -4.4898766213323009
    Empirical Dispersion Energy =           0.0000000000000000
    VV10 Nonlocal Energy =                  0.0000000000000000
    Total Energy =                        -76.3784075995978213



Properties will be evaluated at   0.000000,   0.000000,   0.000000 [a0]

Properties computed using

 the SCF density matrix

  Nuclear Dipole Moment: [e a0]
     X:     0.0000      Y:     0.0000      Z:    -0.9763

  Electronic Dipole Moment: [e a0]
     X:     0.0000      Y:     0.0000      Z:    -0.0368

  Dipole Moment: [e a0]
     X:     0.0000      Y:     0.0000      Z:    -1.0130     Total:     1.0130

  Dipole Moment: [D]
     X:     0.0000      Y:     0.0000      Z:    -2.5748     Total:     2.5748


*** tstop() called on BlauweDraak at Thu Jun  2 11:07:41 2022
Module time:
	user time   =       0.97 seconds =       0.02 minutes
	system time =       0.00 seconds =       0.00 minutes
	total time  =          1 seconds =       0.02 minutes
Total time:
	user time   =       3.04 seconds =       0.05 minutes
	system time =       0.02 seconds =       0.00 minutes
	total time  =          3 seconds =       0.05 minutes


-76.37840759959782

## Referencias

- P. Hohenberg and W. Kohn, Phys. Rev. 136, B864 (1964).
- P. Atkins, Molecular Quantum Mechanics (Oxford University Press, 2005).
- D. Rappoport, N. R. M. Crawford, F. Furche, y K. Burke, Which functional should I choose?, (2008).
- W. Koch y M.C. Holthausen, A Chemist’s Guide to Density Functional Theory, (2001).
- K. Burke y L.O. Wagner, Int. J. Quantum Chem. 113, 96 (2013).
- K. Burke, J. Chem. Phys. 136, (2012).